# Verkehrszählung - Views erstellen

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot')
!pip install seaborn

!pip install folium
import folium # plotting library

## 1) Verkehrszählung

## 1.1) Datensatz einlesen

In [3]:
df_verkehrszaehlung_2020 = pd.read_csv('./data/verkehrszaehlung_2020_mit_quartier_kreis.csv')

In [4]:
df_verkehrszaehlung_2020.shape

(266448, 28)

In [5]:
df_verkehrszaehlung_2020.head()

,MSID,MessungDatum,ZeitKategorie,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungZeit,Koord_WGS84_E,Koord_WGS84_N,Quartier,Kreis
0,Z001M001,2020-01-01,Nacht,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,1061.0,Gemessen,00:00:00,47.341059,8.537068,8038,Kreis 2
1,Z001M001,2020-01-01,Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 07:00:00,2021-02-04,441.0,Gemessen,07:00:00,47.341059,8.537068,8038,Kreis 2
2,Z001M001,2020-01-01,Nachmittag,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 13:00:00,2021-02-04,1241.0,Gemessen,13:00:00,47.341059,8.537068,8038,Kreis 2
3,Z001M001,2020-01-01,Abend,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 19:00:00,2021-02-04,589.0,Gemessen,19:00:00,47.341059,8.537068,8038,Kreis 2
4,Z001M001,2020-01-02,Nacht,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-02 00:00:00,2021-02-04,173.0,Gemessen,00:00:00,47.341059,8.537068,8038,Kreis 2


In [6]:
# Total gemessene Fahrzeuge
df_verkehrszaehlung_2020['AnzFahrzeuge'].sum()

482597380.0

## 1.2) Map Plots

### 1.2.1) Messstationen

In [7]:
df_gr = df_verkehrszaehlung_2020.groupby('ZSName').agg({'ZSName':'first','AnzFahrzeuge':'sum', 'Koord_WGS84_E':'first', 'Koord_WGS84_N':'first'})
real_df = pd.DataFrame(df_gr)

In [8]:
# Farbgruppe nach Unfalltyp
out_lst_grpType = df_gr.AnzFahrzeuge.unique()
out_df_grpType = pd.DataFrame(out_lst_grpType) 
out_df_grpType = out_df_grpType.rename(columns={0: "AnzFahrzeuge"})
out_df_grpType['colorGrp'] = out_df_grpType.apply(lambda x : list(np.random.choice(range(256), size=3)), axis=1)


In [9]:
df_gr.head()

,ZSName,AnzFahrzeuge,Koord_WGS84_E,Koord_WGS84_N
ZSName,,,,
A3W (A4 Uetlibergtunnel Brunau),A3W (A4 Uetlibergtunnel Brunau),10508879.0,47.347115,8.520002
A3W (Chur Brunau),A3W (Chur Brunau),10115643.0,47.347055,8.520122
Albisriederstrasse (Hubertus),Albisriederstrasse (Hubertus),3723119.0,47.377470,8.499435
Albisriederstrasse (Lyrenweg),Albisriederstrasse (Lyrenweg),2548938.0,47.374504,8.482991
Albisstrasse (Widmerstrasse),Albisstrasse (Widmerstrasse),5595914.0,47.334167,8.529921


In [10]:
df_gr = pd.merge(df_gr, out_df_grpType, on='AnzFahrzeuge')

In [11]:
# Zürich lon, lat
latitude = 47.36667
longitude = 8.55

In [12]:
map_markets = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, colo in zip(df_gr['Koord_WGS84_E'], df_gr['Koord_WGS84_N'], df_gr['colorGrp']):
        folium.CircleMarker(
                [lat, lng],
                radius=3,
                fill=True,
                fill_opacity=0.7,
                parse_html = False).add_to(map_markets)  
        
map_markets

### 1.2.2) Anzahl Fahrzeuge pro Kreis

In [14]:
map_path = './data/Quartiergrenzen_ZH/stzh.adm_statistische_quartiere_map.geojson'

In [13]:
x = df_verkehrszaehlung_2020[['Kreis','MSID']].groupby(['Kreis'])
z = x.count().reset_index()

In [16]:
map_markets = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.Choropleth(
    geo_data= map_path,
    name='Number of Cars',
    data=z,
    columns=['Kreis','MSID'],
    key_on='feature.properties.kname',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    legend_name='Number of Cars'
).add_to(map_markets) 

folium.LayerControl().add_to(map_markets)

map_markets